In [1]:
# Activate project enviroment 
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Users\Herbert\Documents\GitHub\AMO_Indivdiual_Project`


In [2]:
using JuMP
using Plots
using XLSX
using DataFrames
using Complementarity
using Ipopt
using Distributions
using CSV
using HiGHS
using StatsPlots

## Deterministic single-node dynamic GEP

In [3]:
#Read in demand options for target year (Output of Clustering.ipynb) and sort by powerlevel (ascending)
options = sort!(CSV.read("./data/options.csv", DataFrame),:powerlevel)

Row,powerlevel,weight
,Float64,Float64
1,0.77741,0.244064
2,0.932355,0.280023
3,1.09936,0.313242
4,1.25909,0.162671


In [39]:
# Indexes
G = 1:12 # Existing generators
C = 1:5 # Candidate generators
D = 1:1 # Demands (sum of all Loads, because single node Model)
O = 1:4 # Options
Q = 1:7 # Number of steps for installable capacity. (Same for each candidate)
T = 1:4 # Time steps where new Capacity can be installed

[2,2,2,2,2,1]

# Parameters
Ic = [600_000, 700_000, 400_000, 1_200_000, 950_000] #Total Investment cost per installed MW for each candidate
a = [0.15,0.1,0.5,0.03] # amortization rates for each timestep
pd = sum([84,75,139,58,55,106,97,132,135,150,205,150,245,77,258,141,100]) .* options.powerlevel # demand for each option
Emax = [106.4,106.4,245,413.7,42,108.5,108.5,280,280,210,217,245] # existing generator maximium capacity
Ec = [13.32,13.32,20.7,20.93,26.11,10.52,10.52,6.02,5.47,7,10.52,10.89] # existing Generator Energy cost
CandiData = [0.0 30.0 60.0 90.0 120.0 150.0 180.0  # Possible Capacity installation levels for each candidate
             0.0 30.0 60.0 90.0 120.0 150.0 180.0
             0.0 40.0 80.0 120.0 160.0 200.0 240.0
             0.0 40.0 80.0 120.0 160.0 200.0 240.0
             0.0 50.0 100.0 150.0 200.0 250.0 300.0]
Cc = [11.7,10.4,13.2,8.7,9.5] # operating cost per MW for candidates
vLOL = 500 # value of lost load
wp = sum([120.54,115.52,53.34,38.16]) # wind power (no cost)

ρ = Integer.(round.(options.weight .* 8760)) # = [2744,1425,2453,2138] -> hours in a year for each option

M = 1_000_000 # Big M 

# one last thing - we consider the demands as historical data, but demand in the future will grow. So, we will increase the demand options by a percentage for each year at each timestep.
# Our planning horzion is 20 years. To get realistic average annual increase in demand,
# we use historical data of the yearly demand from 2015 to 2018 in Germany source: https://www.smard.de/home/downloadcenter/download-marktdaten/.

yearly_demands = [500219502.50, 503083631.75, 505674706.50 ,509156603.75] 
annual_increase = mean([yearly_demands[i+1]/yearly_demands[i] for i in 1:3]) # calculate the mean of the relative annual changes from 2015-2018
increases = [annual_increase^5,annual_increase^10,annual_increase^15,annual_increase^20] #get increase factor for each timestep
pd = [round.(pd.*increases[i],digits = 1) for i in T] # multiply demands by expected increase factors for 5,10,15 and 20 years.
ρ

4-element Vector{Int64}:
 2138
 2453
 2744
 1425

In [40]:
#Building the MILP Model (as explained in "Investment in Electricity Generation and Transmission", Conejo et al, Chapter 3.3.3)

gep1 = Model(HiGHS.Optimizer)
set_silent(gep1)

@variables(gep1, begin
        pge[g in G, o in O,t in T] >= 0
        pgc[c in C,o in O,t in T] >= 0
        pcmax[c in C,t in T] >= 0
        λ[o in O,t in T] >= 0
        uOp[c in C, q in Q,t in T], Bin
        μ_emax[g in G, o in O,t in T] >= 0
        μ_cmax[c in C, o in O, t in T] >= 0
        zaux[c in C,q in Q, o in O, t in T]
        zaux2[c in C, q in Q, o in O, t in T]
        Lshed[o in O,t in T] >= 0
end)

@constraint(gep1, EQ7B[c in C,t in T], pcmax[c,t] == sum(uOp[c,q,t]*CandiData[c,q] for q in Q))

@constraint(gep1, EQ7C[c in C,t in T], sum(uOp[c,:,t]) == 1)

@constraint(gep1, EQ7E[o in O, t in T], sum(pge[g,o,t] for g in G) + sum(pgc[c,o,t] for c in C) + wp + Lshed[o,t] == pd[t][o])

@constraint(gep1,EQ7F[g in G, o in O,t in T], pge[g,o,t] <= Emax[g])

# -> power generation of candidate c cannot be higher than sum of installed capacities of present and past timesteps
@constraint(gep1, EQ7G[c in C, o in O, t in T],pgc[c,o,t] <= sum(pcmax[c,1:t])) 

@constraint(gep1, EQ7H[g in G, o in O,t in T], Ec[g] - λ[o,t] + μ_emax[g,o,t] >= 0)

@constraint(gep1, EQ7I[c in C, o in O, t in T], Cc[c] - λ[o,t] + μ_cmax[c,o,t] >= 0)

@constraint(gep1, EQ7L[o in O,t in T], sum(Ec[g] * pge[g,o,t] for g in G) + sum(Cc[c] * pgc[c,o,t] for c in C) ==
                λ[o,t] * pd[t][o] - sum(μ_emax[g,o,t] * Emax[g] for g in G) - sum(sum(zaux[c,q,o,t] for q in Q) for c in C ))

@constraint(gep1, EQ7M[c in C, q in Q, o in O,t in T], zaux[c,q,o,t] == μ_cmax[c,o,t]* CandiData[c,q] - zaux2[c,q,o,t])

@constraint(gep1, EQ7Na[c in C, q in Q, o in O, t in T], 0 <= zaux[c,q,o,t])

@constraint(gep1, EQ7Nb[c in C, q in Q, o in O, t in T], zaux[c,q,o,t] <= uOp[c,q,t] * M)

@constraint(gep1, EQ7Oa[c in C, q in Q, o in O, t in T], 0 <= zaux2[c,q,o,t])

@constraint(gep1, EQ7Ob[c in C, q in Q, o in O, t in T], zaux2[c,q,o,t] <= (1-uOp[c,q,t]) * M)

@objective(gep1, Min, sum(sum(ρ[o] * (sum(Ec[g]* pge[g,o,t] for g in G) + sum(Cc[c]*pgc[c,o,t] for c in C) + vLOL*Lshed[o,t]) for o in O) + sum(a[t] * Ic[c] * pcmax[c,t] for c in C) for t in T))

28478.16 pge[1,1,1] + 28478.16 pge[2,1,1] + 44256.6 pge[3,1,1] + 44748.34 pge[4,1,1] + 55823.18 pge[5,1,1] + 22491.76 pge[6,1,1] + 22491.76 pge[7,1,1] + 12870.759999999998 pge[8,1,1] + 11694.859999999999 pge[9,1,1] + 14966 pge[10,1,1] + 22491.76 pge[11,1,1] + 23282.82 pge[12,1,1] + 25014.6 pgc[1,1,1] + 22235.2 pgc[2,1,1] + 28221.6 pgc[3,1,1] + 18600.6 pgc[4,1,1] + 20311 pgc[5,1,1] + 1069000 Lshed[1,1] + 32673.96 pge[1,2,1] + 32673.96 pge[2,2,1] + 50777.1 pge[3,2,1] + 51341.29 pge[4,2,1] + 64047.83 pge[5,2,1] + 25805.559999999998 pge[6,2,1] + 25805.559999999998 pge[7,2,1] + 14767.06 pge[8,2,1] + 13417.91 pge[9,2,1] + 17171 pge[10,2,1] + 25805.559999999998 pge[11,2,1] + 26713.170000000002 pge[12,2,1] + 28700.1 pgc[1,2,1] + 25511.2 pgc[2,2,1] + 32379.6 pgc[3,2,1] + 21341.1 pgc[4,2,1] + 23303.5 pgc[5,2,1] + 1226500 Lshed[2,1] + 36550.08 pge[1,3,1] + 36550.08 pge[2,3,1] + 56800.799999999996 pge[3,3,1] + 57431.92 pge[4,3,1] + 71645.84 pge[5,3,1] + 28866.879999999997 pge[6,3,1] + 28866.879999

In [41]:
optimize!(gep1)
termination_status(gep1)

OPTIMAL::TerminationStatusCode = 1

In [42]:
value.(pgc)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:4
    Dimension 3, 1:4
And data, a 5×4×4 Array{Float64, 3}:
[:, :, 1] =
   0.0    0.0    0.0    0.0
 180.0  180.0  180.0  180.0
   0.0    0.0  120.0  120.0
   0.0    0.0    0.0    0.0
 200.0  200.0  200.0  200.0

[:, :, 2] =
   0.0    0.0    0.0    0.0
 360.0  360.0  360.0  360.0
   0.0    0.0  120.0  120.0
   0.0    0.0    0.0    0.0
 200.0  200.0  200.0  200.0

[:, :, 3] =
   0.0    0.0    0.0    0.0
 360.0  360.0  360.0  360.0
   0.0    0.0  120.0  120.0
   0.0    0.0    0.0    0.0
 200.0  200.0  200.0  200.0

[:, :, 4] =
   0.0     0.0    0.0     0.0
 333.24  450.0  450.0   450.0
   0.0     0.0    3.74  120.0
  -0.0    -0.0   -0.0    -0.0
 500.0   500.0  500.0   500.0

In [33]:
value.(uOp)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:7
    Dimension 3, 1:4
And data, a 5×7×4 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0

[:, :, 3] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 1.0   0.0  0.0  0.0   0.0  0.0  0.0
 0.0   0.0  0.0  0.0  -0.0  1.0  0.0
 1.0   0.0  0.0  0.0   0.0  0.0  0.0
 1.0  -0.0  0.0  0.0   0.0  0.0  0.0
 0.0   0.0  0.0  0.0   0.0  0.0  1.0

In [38]:
value.(pcmax)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:4
And data, a 5×4 Matrix{Float64}:
   0.0    0.0  0.0    0.0
 180.0  180.0  0.0   90.0
 120.0    0.0  0.0    0.0
   0.0    0.0  0.0    0.0
 200.0    0.0  0.0  300.0